In [1]:
import pandas as pd
import numpy as np
import data_preprocessing as dp
import mdp
import Replay_Buffer
import torch as T

### load training data

In [2]:
# load data: DR_USA_Roundabout_FT_train.csv

# Roundabout data google drive path: "https://drive.google.com/file/d/1Ls92QosBDIyoDcEudSVGukP68pw1FRXW/view?usp=sharing"
# please download it to your local computer

url_FT_train = 'INTERACTION/DR_USA_Roundabout_FT_train.csv'
df_train = pd.read_csv(url_FT_train)

# look at the top 30 rows of data
df_train.head(30)

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width
0,1.0,1,1,100,pedestrian/bicycle,1044.509,1012.175,0.103,-1.005,NaN,NaN,NaN
1,1.0,2,1,100,pedestrian/bicycle,1043.840,1013.348,-0.101,-1.362,NaN,NaN,NaN
2,1.0,3,1,100,pedestrian/bicycle,1007.638,1022.038,0.687,0.787,NaN,NaN,NaN
3,1.0,4,1,100,car,1036.888,976.415,-1.063,1.980,2.064,5.20,1.86
4,1.0,5,1,100,car,1038.206,993.477,-2.343,5.161,1.997,4.68,1.90
5,1.0,6,1,100,car,1004.449,984.315,1.823,-0.369,-0.200,4.93,1.97
6,1.0,7,1,100,car,1013.407,987.769,4.564,-3.386,-0.638,4.83,1.93
7,1.0,8,1,100,car,1048.917,980.407,-0.894,0.591,2.557,5.13,1.96
8,1.0,9,1,100,car,993.728,985.780,1.184,0.012,0.010,4.54,1.81
9,1.0,10,1,100,car,981.026,985.788,4.411,0.197,0.045,4.55,1.78


### 1. create unique_id
### 2. change length, width and psi_rad from NaN to 0 for pedestrain/bicycle
### 3. change agent type to number for state vector (pedestrian/bicycle = -1; car = 1)

In [3]:
# add a new label unique_id using case_id and track_id
df_train['unique_id'] = df_train['case_id'].astype(int) * 100+ df_train['track_id']

# change length, width and psi_rad from NaN to 0 for pedestrain/bicycle
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'length'] = 0
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'width'] = 0
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'psi_rad'] = 0


### important!!!
# agent_type pedestrain/bicycle = -1
# agent_type car = 1
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'agent_type'] = -1
df_train.loc[df_train['agent_type'] == 'car', 'agent_type'] = 1
df_train

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width,unique_id
0,1.0,1,1,100,-1,1044.509,1012.175,0.103,-1.005,0.000,0.00,0.00,101
1,1.0,2,1,100,-1,1043.840,1013.348,-0.101,-1.362,0.000,0.00,0.00,102
2,1.0,3,1,100,-1,1007.638,1022.038,0.687,0.787,0.000,0.00,0.00,103
3,1.0,4,1,100,1,1036.888,976.415,-1.063,1.980,2.064,5.20,1.86,104
4,1.0,5,1,100,1,1038.206,993.477,-2.343,5.161,1.997,4.68,1.90,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4243282,9811.0,8,40,4000,1,1022.107,985.137,7.470,1.627,0.215,4.19,1.87,981108
4243283,9811.0,9,40,4000,1,996.468,986.201,7.374,-0.178,-0.024,4.64,1.90,981109
4243284,9811.0,10,40,4000,1,1019.894,1009.985,-7.780,-0.610,-3.063,4.72,1.87,981110
4243285,9811.0,11,40,4000,1,1028.592,1022.465,-1.619,-1.729,-2.324,4.49,1.83,981111


### use 2 cases (case1 and case2) for demonstrating and testing

In [4]:
# use case 1 and case 2 as a toy test dataframe
df_train_small = df_train[df_train['case_id'] <= 2.0]
df_train_small 

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width,unique_id
0,1.0,1,1,100,-1,1044.509,1012.175,0.103,-1.005,0.000,0.00,0.00,101
1,1.0,2,1,100,-1,1043.840,1013.348,-0.101,-1.362,0.000,0.00,0.00,102
2,1.0,3,1,100,-1,1007.638,1022.038,0.687,0.787,0.000,0.00,0.00,103
3,1.0,4,1,100,1,1036.888,976.415,-1.063,1.980,2.064,5.20,1.86,104
4,1.0,5,1,100,1,1038.206,993.477,-2.343,5.161,1.997,4.68,1.90,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,2.0,8,40,4000,1,1008.481,993.097,3.799,-4.802,-0.901,4.80,1.90,208
878,2.0,9,40,4000,1,1044.608,984.263,-0.540,0.413,2.489,4.48,1.80,209
879,2.0,10,40,4000,1,990.708,985.918,4.940,0.168,0.034,4.61,1.86,210
880,2.0,11,40,4000,1,1036.495,974.560,-2.085,2.679,2.232,4.71,1.89,211


In [22]:
df_train.unique_id.nunique()

129019

### create uniqueTracks (a dictionary strore every unique agents (car, bike, or pedestrain)

In [49]:
# load uniqueTracks data
# uniqueTracks is a dictionary strore each unique object
uniqueTracks = dp.read_uniqueTracks(df_train_small)
uniqueTracks

{101: <data_preprocessing.uniqueTrack at 0x11ab7fc90>,
 102: <data_preprocessing.uniqueTrack at 0x37b3c4f10>,
 103: <data_preprocessing.uniqueTrack at 0x37b3c2fd0>,
 104: <data_preprocessing.uniqueTrack at 0x11ab7f5d0>,
 105: <data_preprocessing.uniqueTrack at 0x37b3c3f90>,
 106: <data_preprocessing.uniqueTrack at 0x37b7f0750>,
 107: <data_preprocessing.uniqueTrack at 0x37b3c0a10>,
 108: <data_preprocessing.uniqueTrack at 0x37b7f1a90>,
 109: <data_preprocessing.uniqueTrack at 0x37b3c3550>,
 110: <data_preprocessing.uniqueTrack at 0x37c587d10>,
 111: <data_preprocessing.uniqueTrack at 0x37b3c3b90>,
 112: <data_preprocessing.uniqueTrack at 0x37b3c1350>,
 113: <data_preprocessing.uniqueTrack at 0x37b3c23d0>,
 114: <data_preprocessing.uniqueTrack at 0x37b3c2010>,
 115: <data_preprocessing.uniqueTrack at 0x37c407250>,
 201: <data_preprocessing.uniqueTrack at 0x37c55c1d0>,
 202: <data_preprocessing.uniqueTrack at 0x37c54c2d0>,
 203: <data_preprocessing.uniqueTrack at 0x37c674a10>,
 204: <dat

### information stored in the object

In [6]:
# take a look at the first object
object_1 = uniqueTracks[101]
print(object_1)

uniqueTrack: case_id=1.0, track_id=1,                 unique_id=101, agent_type=-1,                 length=0.0, width=0.0
    100: {'agent_type': -1, 'x': 1044.509, 'y': 1012.175, 'vx': 0.103, 'vy': -1.005, 'psi_rad': 0.0}
    200: {'agent_type': -1, 'x': 1044.52, 'y': 1012.075, 'vx': 0.116, 'vy': -1.0, 'psi_rad': 0.0}
    300: {'agent_type': -1, 'x': 1044.533, 'y': 1011.974, 'vx': 0.139, 'vy': -1.012, 'psi_rad': 0.0}
    400: {'agent_type': -1, 'x': 1044.548, 'y': 1011.872, 'vx': 0.169, 'vy': -1.042, 'psi_rad': 0.0}
    500: {'agent_type': -1, 'x': 1044.567, 'y': 1011.765, 'vx': 0.201, 'vy': -1.088, 'psi_rad': 0.0}
    600: {'agent_type': -1, 'x': 1044.588, 'y': 1011.654, 'vx': 0.233, 'vy': -1.143, 'psi_rad': 0.0}
    700: {'agent_type': -1, 'x': 1044.613, 'y': 1011.536, 'vx': 0.261, 'vy': -1.204, 'psi_rad': 0.0}
    800: {'agent_type': -1, 'x': 1044.64, 'y': 1011.413, 'vx': 0.281, 'vy': -1.268, 'psi_rad': 0.0}
    900: {'agent_type': -1, 'x': 1044.669, 'y': 1011.283, 'vx': 0.288, 'vy

In [7]:
print('Object type:', object_1.agent_type)
print('Case ID:', object_1.case_id)
print('Object length:', object_1.length)
print('Record once every 0.1 second, total recording time is 4 seconds-> the length of data:', len(object_1.motionState))
print('motion state at 1st 0.1 second', object_1.motionState[100])
print('motion state at 2nd 0.1 second', object_1.motionState[200])
print()

Object type: -1
Case ID: 1.0
Object length: 0.0
Record once every 0.1 second, total recording time is 4 seconds-> the length of data: 40
motion state at 1st 0.1 second {'agent_type': -1, 'x': 1044.509, 'y': 1012.175, 'vx': 0.103, 'vy': -1.005, 'psi_rad': 0.0}
motion state at 2nd 0.1 second {'agent_type': -1, 'x': 1044.52, 'y': 1012.075, 'vx': 0.116, 'vy': -1.0, 'psi_rad': 0.0}



## look at a car agent

In [8]:
object_2 = uniqueTracks[104]
print('Object type:', object_2.agent_type)
print('Object length:', object_2.length)
print('Record once every 0.1 second, total recording time is 4 seconds-> the length of data:', len(object_2.motionState))
print('motion state at 1st 0.1 second', object_2.motionState[100])
print('motion state at 2nd 0.1 second', object_2.motionState[200])

Object type: 1
Object length: 5.2
Record once every 0.1 second, total recording time is 4 seconds-> the length of data: 40
motion state at 1st 0.1 second {'agent_type': 1, 'x': 1036.888, 'y': 976.415, 'vx': -1.063, 'vy': 1.98, 'psi_rad': 2.064}
motion state at 2nd 0.1 second {'agent_type': 1, 'x': 1036.793, 'y': 976.619, 'vx': -1.047, 'vy': 1.985, 'psi_rad': 2.056}


### get other/interative vehicles in the same case and in the same time frame 

In [9]:
ego_id = 104
current_time = 100

# get other vehicles with same case and time
other_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)
print("other vehicles:", other_agents)
print("interactive vehicles:", interactive_agents)

other vehicles: [101, 102, 103, 105, 106, 107, 108, 109, 110, 111, 112, 113]
interactive vehicles: [108, 105, 111]


### get MDP tuple (State, Action, Reware, Next State)

In [10]:
s, a, r, s_next, the_last_episode = mdp.MDP_tuple(ego_id, interactive_agents, current_time, uniqueTracks)

# s = [ego_id, int_1_id, int_2_id, int_3_id, ego_features, int_1_features, int_2_features, int_3_features]
# a = [vx, vy] at next timestep
# r = TBD
# s_next = [ego_id, int_1_id, int_2_id, int_3_id, ego_features, int_1_features, int_2_features, int_3_features] at next timestep
# the_last_episode = True if it's the last episode

In [11]:
s

[104,
 108,
 105,
 111,
 1,
 1036.888,
 976.415,
 -1.063,
 1.98,
 2.064,
 1,
 1048.917,
 980.407,
 -0.894,
 0.591,
 2.557,
 1,
 1038.206,
 993.477,
 -2.343,
 5.161,
 1.997,
 1,
 1055.975,
 975.473,
 -1.121,
 0.764,
 2.543]

In [12]:
a

[-1.047, 1.985]

In [13]:
s_next

[104,
 108,
 105,
 111,
 1,
 1036.793,
 976.619,
 -1.047,
 1.985,
 2.056,
 1,
 1048.827,
 980.465,
 -0.92,
 0.608,
 2.558,
 1,
 1037.975,
 993.996,
 -2.341,
 5.201,
 1.994,
 1,
 1055.864,
 975.551,
 -1.13,
 0.773,
 2.542]

In [14]:
the_last_episode

False

In [15]:
s, a, r, s_next, the_last_episode = mdp.MDP_tuple(ego_id, interactive_agents, 3900, uniqueTracks)

In [16]:
the_last_episode

True

In [60]:
#Set Device
device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')

In [61]:
#Initialize the buffer
replay_buffer = Replay_Buffer.ReplayBuffer(28, 2, device, 100000)

In [65]:
# Loop to add each transition to the replay buffer
# Once all added we can save and then load at time of training
for unique_id,group in df_train_small.groupby('unique_id'):
    timestamps = group['timestamp_ms'].unique()
    num_timestamps = len(timestamps)
    for timestamp in timestamps[:num_timestamps - 1]:
        frame = group[group['timestamp_ms'] == timestamp]
        ego_id = unique_id
        _, interactive_agents = mdp.get_other_agents_unique_id(ego_id, timestamp, uniqueTracks)
        s, a, r, s_next, the_last_episode = mdp.MDP_tuple(ego_id, interactive_agents, timestamp, uniqueTracks)
        replay_buffer.add(s, a, s_next, r, the_last_episode)
        i=i+1

print(i) #Total transitions we should have !!!

855


In [79]:
#Sample the Buffer
test = replay_buffer.sample(2)
test

(tensor([[ 2.0700e+02,  2.0600e+02,  2.0900e+02,  2.0500e+02,  1.0000e+00,
           1.0394e+03,  1.0105e+03,  1.6540e+00,  7.1460e+00,  1.3430e+00,
           1.0000e+00,  1.0394e+03,  9.9771e+02,  3.7780e+00,  4.4010e+00,
           8.6100e-01,  1.0000e+00,  1.0450e+03,  9.8398e+02, -6.6600e-01,
           5.0000e-01,  2.4980e+00,  1.0000e+00,  1.0354e+03,  9.8081e+02,
          -8.0700e-01,  3.1720e+00,  1.8200e+00],
         [ 1.1000e+02,  1.0900e+02,  1.1300e+02,  1.0600e+02,  1.0000e+00,
           9.8377e+02,  9.8587e+02,  3.2410e+00,  9.8000e-02,  3.0000e-02,
           1.0000e+00,  9.9446e+02,  9.8578e+02,  9.5700e-01,  6.0000e-03,
           7.0000e-03,  1.0000e+00,  9.6930e+02,  9.8478e+02,  4.9470e+00,
           3.1100e-01,  6.3000e-02,  1.0000e+00,  1.0060e+03,  9.8397e+02,
           2.7910e+00, -5.9100e-01, -2.0900e-01]]),
 tensor([[1.8200, 7.2200],
         [3.0670, 0.0870]]),
 tensor([[ 2.0700e+02,  2.0600e+02,  2.0900e+02,  2.0500e+02,  1.0000e+00,
           1.0396

In [72]:
#Save the Buffer
replay_buffer.save("buffers/DR_USA_Roundabout_FT")

In [77]:
#Load the Buffer
replay_buffer_load = Replay_Buffer.ReplayBuffer(28, 2, device, 100000)
replay_buffer_load.load(f"buffers/DR_USA_Roundabout_FT")

In [78]:
replay_buffer_load.sample(2)

(tensor([[ 2.0800e+02,  2.0100e+02,  2.0600e+02,  2.0700e+02,  1.0000e+00,
           1.0013e+03,  1.0095e+03,  1.5400e+00, -5.4680e+00, -1.2960e+00,
           1.0000e+00,  9.7508e+02,  9.9325e+02, -1.0575e+01, -2.0050e+00,
          -2.9540e+00,  1.0000e+00,  1.0296e+03,  9.8698e+02,  5.1060e+00,
           3.4460e+00,  5.9400e-01,  1.0000e+00,  1.0387e+03,  9.9672e+02,
          -9.2200e-01,  4.0650e+00,  1.7940e+00],
         [ 2.0300e+02,  2.0700e+02,  2.0600e+02,  2.0900e+02,  1.0000e+00,
           1.0431e+03,  1.0024e+03,  6.0680e+00,  2.8400e+00,  4.3800e-01,
           1.0000e+00,  1.0386e+03,  9.9713e+02, -8.8100e-01,  4.2080e+00,
           1.7770e+00,  1.0000e+00,  1.0300e+03,  9.8735e+02,  5.0150e+00,
           3.5730e+00,  6.1900e-01,  1.0000e+00,  1.0504e+03,  9.8003e+02,
          -4.7550e+00,  3.3810e+00,  2.5230e+00]]),
 tensor([[ 1.5310, -5.5390],
         [ 6.1140,  2.7270]]),
 tensor([[ 2.0800e+02,  2.0100e+02,  2.0600e+02,  2.0700e+02,  1.0000e+00,
           1.